There's some error in this package, it only works well on google colab(or other linux systems).

In [ ]:
!pip install -q numpy
!apt-get install ffmpeg
!apt-get install espeak
!pip install -q beautifulsoup4
!pip install -q lxml
!apt-get install libgdal-dev

In [ ]:
!wget https://raw.githubusercontent.com/readbeyond/aeneas/master/install_dependencies.sh
!bash install_dependencies.sh

In [ ]:
!git clone https://github.com/ReadBeyond/aeneas.git
!cd /content/aeneas
!sudo pip install -r /content/aeneas/requirements.txt
!python /content/aeneas/setup.py build_ext --inplace
!python /content/aeneas/aeneas_check_setup.py
!pip install -q aeneas

In [ ]:
!python -m aeneas.diagnostics

In [10]:
import aeneas.globalconstants as gc
from aeneas.executetask import ExecuteTask
from aeneas.task import Task
import json

def format_time(aeneas_time):
    """ Converts aeneas time format to SRT time format """
    hours, remainder = divmod(float(aeneas_time), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02}:{int(minutes):02}:{seconds:06.3f}".replace('.', ',')

def process_aeneas_output(sync_map_file):
    """Process the output from aeneas to create an SRT file."""
    with open(sync_map_file, 'r') as f:
        sync_map = json.load(f)

    srt_template = "{index}\n{start} --> {end}\n{text}\n\n"
    index = 1
    with open('subtitles.srt', 'w') as srt_file:
        for fragment in sync_map['fragments']:
            # Replace <eol> with a newline for two-line display, and remove <eob> tags
            text = fragment['lines'][0].replace('<eol> ', '\n').replace('<eob>', '').strip()
            start = format_time(fragment['begin'])
            end = format_time(fragment['end'])
            srt_file.write(srt_template.format(index=index, start=start, end=end, text=text))
            index += 1

def main():
    # Initialize the file name
    audio_file = ""
    text_file = ""
    # Create a Task object
    config_string = u"task_language=eng|is_text_type=plain|os_task_file_format=json"
    task = Task(config_string=config_string)
    task.text_file_path_absolute = f"/content/{text_file}.txt"
    task.audio_file_path_absolute = f"/content/{audio_file}.mp3"
    task.sync_map_file_path_absolute = "/content/syncmap.json"

    # Process the task
    ExecuteTask(task).execute()

    # Output the sync map to a file and process it
    task.output_sync_map_file()

    # Process the aeneas output to create an SRT file
    process_aeneas_output(task.sync_map_file_path_absolute)

if __name__ == "__main__":
    main()
